In [ ]:
from vnstock import Company
from vnstock import Listing
from vnstock import Finance
company = Company(symbol = "VTO", source="TCBS")
company2 = Company(symbol = "MBS", source="VCI")
listing = Listing()
finance = Finance(symbol="VTO", source="VCI")



In [6]:
import pandas as pd
# Cấu hình để hiển thị tất cả các cột mà không bị rút gọn
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [9]:
listing.industries_icb()
company2.shareholders()
finance.balance_sheet().head(5)

,ticker,yearReport,lengthReport,CURRENT ASSETS (Bn. VND),Cash and cash equivalents (Bn. VND),Short-term investments (Bn. VND),Accounts receivable (Bn. VND),Net Inventories,Other current assets,LONG-TERM ASSETS (Bn. VND),Fixed assets (Bn. VND),Long-term investments (Bn. VND),Other non-current assets,TOTAL ASSETS (Bn. VND),LIABILITIES (Bn. VND),Current liabilities (Bn. VND),Long-term liabilities (Bn. VND),OWNER'S EQUITY(Bn.VND),Capital and reserves (Bn. VND),Undistributed earnings (Bn. VND),TOTAL RESOURCES (Bn. VND),Prepayments to suppliers (Bn. VND),"Inventories, Net (Bn. VND)",Other current assets (Bn. VND),Investment and development funds (Bn. VND),Common shares (Bn. VND),Paid-in capital (Bn. VND),Long-term borrowings (Bn. VND),Advances from customers (Bn. VND),Short-term borrowings (Bn. VND),Long-term prepayments (Bn. VND),Other long-term assets (Bn. VND),Other long-term receivables (Bn. VND),Long-term trade receivables (Bn. VND)
0,VTO,2025,2,1061952147417,884407077009,0,83660050689,92675789277,1209230442,453507002351,431748561183,10741176461,10751764707,1515459149768,402408564784,293327259221,109081305563,1113050584984,1113050584984,100768812695,1515459149768,5318197113,92675789277,0,213491855223,798666660000,798666660000,109081305563,123693634,59384425375,0,64500000000,265500000,265500000
1,VTO,2025,1,1073396047334,890256418825,0,99849433644,83199941815,90253050,487213278925,465454837757,10741176461,10751764707,1560609326259,371116303298,243863347798,127252955500,1189493022961,1189493022961,177211250672,1560609326259,4001948723,83199941815,0,213491855223,798666660000,798666660000,127252955500,121059853,60492584875,0,64500000000,265500000,265500000
2,VTO,2024,4,1085758815951,890556025885,0,101908297705,91784230904,1510261457,530069039831,508310598663,10741176461,10751764707,1615827855782,445618408850,299974707394,145643701456,1170209446932,1170209446932,157927674643,1615827855782,5514408572,91784230904,0,213491855223,798666660000,798666660000,145643701456,120170952,71380815025,0,64500000000,265500000,265500000
3,VTO,2024,3,1055491225912,848376702320,0,114856175336,91150183235,1108165021,601051560671,579345064583,10676245111,10764750977,1656542786583,520685065005,357985263755,162699801250,1135857721578,1135857721578,123575949289,1656542786583,10020646581,91150183235,0,213491855223,798666660000,798666660000,162699801250,116356673,66025722625,0,64500000000,265500000,265500000
4,VTO,2024,2,965073521443,788940494225,0,85027667156,90008245607,1097114455,634405098070,612698601982,10676245111,10764750977,1599478619513,483041420046,300283264065,182758155981,1116437199467,1116437199467,104155427178,1599478619513,8889176702,90008245607,0,213491855223,798666660000,798666660000,182758155981,119804104,63506341206,0,64500000000,265500000,265500000


In [8]:
finance.balance_sheet().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 34 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   ticker                                      50 non-null     object
 1   yearReport                                  50 non-null     int64 
 2   lengthReport                                50 non-null     int64 
 3   CURRENT ASSETS (Bn. VND)                    50 non-null     int64 
 4   Cash and cash equivalents (Bn. VND)         50 non-null     int64 
 5   Short-term investments (Bn. VND)            50 non-null     int64 
 6   Accounts receivable (Bn. VND)               50 non-null     int64 
 7   Net Inventories                             50 non-null     int64 
 8   Other current assets                        50 non-null     int64 
 9   LONG-TERM ASSETS (Bn. VND)                  50 non-null     int64 
 10  Fixed assets (Bn. VND)      

In [26]:
# Kiểm tra columns của shareholders data
shareholders_df = company2.shareholders()
print("Shareholders DataFrame columns:")
print(shareholders_df.columns.tolist())
print("\nFirst few rows:")
print(shareholders_df.head())

Shareholders DataFrame columns:
['id', 'share_holder', 'quantity', 'share_own_percent', 'update_date']

First few rows:
         id                           share_holder   quantity  \
0  88114420  Ngân hàng Thương mại Cổ phần Quân đội  437357236   
1  88113275                            Lê Viết Hải     957667   
2  88117423                       Nguyễn Minh Hằng     511201   
3  88113242                            Lê Văn Tuấn     487985   
4  88101432                      Bùi Thị Hồng Hạnh      55581   

   share_own_percent update_date  
0             0.7636  2025-09-03  
1             0.0017  2025-07-25  
2             0.0009  2025-07-25  
3             0.0009  2025-07-25  
4             0.0005  2025-01-23  


In [27]:
# Debug update_date values
df = company2.shareholders()
print("Original update_date values:")
print(df['update_date'].head())
print(f"Type: {type(df['update_date'].iloc[0])}")

# Test safe_date_passthrough function
from apps.stock.utils.safe import safe_date_passthrough
test_date = df['update_date'].iloc[0]
print(f"\nOriginal value: {test_date}")
print(f"After safe_date_passthrough: {safe_date_passthrough(test_date)}")
print(f"Type after: {type(safe_date_passthrough(test_date))}")

Original update_date values:
0    2025-09-03
1    2025-07-25
2    2025-07-25
3    2025-07-25
4    2025-01-23
Name: update_date, dtype: object
Type: <class 'str'>

Original value: 2025-09-03
After safe_date_passthrough: 2025-09-03
Type after: <class 'str'>


In [28]:
# Test updated safe_date_passthrough function
import importlib
import apps.stock.utils.safe
importlib.reload(apps.stock.utils.safe)

from apps.stock.utils.safe import safe_date_passthrough

test_date = "2025-09-03"
result = safe_date_passthrough(test_date)
print(f"Original: {test_date} (type: {type(test_date)})")
print(f"Result: {result} (type: {type(result)})")

# Test với date object
from datetime import date
test_date2 = date(2025, 9, 3)
result2 = safe_date_passthrough(test_date2)
print(f"Date object: {test_date2} (type: {type(test_date2)})")
print(f"Result: {result2} (type: {type(result2)})")

Original: 2025-09-03 (type: <class 'str'>)
Result: 2025-09-03 (type: <class 'datetime.date'>)
Date object: 2025-09-03 (type: <class 'datetime.date'>)
Result: 2025-09-03 (type: <class 'datetime.date'>)


In [30]:
# Test _map_shareholders logic trực tiếp
shareholders_df = company2.shareholders()

# Simulate _map_shareholders logic
def safe_str(value, default=""):
    try:
        if value is None:
            return default
        return str(value)
    except Exception:
        return default

def safe_int(value, default=0):
    try:
        if value is None:
            return default
        return int(value)
    except Exception:
        return default

def safe_decimal(value, default=0.0):
    try:
        if value is None:
            return default
        return float(value)
    except Exception:
        return default

# Test với một row
test_row = shareholders_df.iloc[0]
print("Test row data:")
print(f"share_holder: {test_row['share_holder']}")
print(f"quantity: {test_row['quantity']}")
print(f"share_own_percent: {test_row['share_own_percent']}")
print(f"update_date: {test_row['update_date']} (type: {type(test_row['update_date'])})")

# Test safe functions
mapped_row = {
    "share_holder": safe_str(test_row.get("share_holder")),
    "quantity": safe_int(test_row.get("quantity")),
    "share_own_percent": safe_decimal(test_row.get("share_own_percent")),
    "update_date": safe_date_passthrough(test_row.get("update_date")),
}

print("\nMapped row:")
for key, value in mapped_row.items():
    print(f"{key}: {value} (type: {type(value)})")

Test row data:
share_holder: Ngân hàng Thương mại Cổ phần Quân đội
quantity: 437357236
share_own_percent: 0.7636
update_date: 2025-09-03 (type: <class 'str'>)

Mapped row:
share_holder: Ngân hàng Thương mại Cổ phần Quân đội (type: <class 'str'>)
quantity: 437357236 (type: <class 'int'>)
share_own_percent: 0.7636 (type: <class 'float'>)
update_date: 2025-09-03 (type: <class 'datetime.date'>)


In [15]:
df[df['exchange'] == 'HSX']

,symbol,exchange,type,organ_short_name,organ_name,product_grp_id
1,YEG,HSX,STOCK,Tập đoàn Yeah1,Công ty Cổ phần Tập đoàn Yeah1,STO
2,YBM,HSX,STOCK,Khoáng sản CN Yên Bái,Công ty Cổ phần Khoáng sản Công nghiệp Yên Bái,STO
33,VTP,HSX,STOCK,Bưu chính Viettel,Tổng Công ty Cổ phần Bưu chính Viettel,STO
34,VTO,HSX,STOCK,VITACO,Công ty Cổ phần Vận tải Xăng dầu VITACO,STO
45,VTB,HSX,STOCK,Viettronics Tân Bình,Công ty Cổ phần Viettronics Tân Bình,STO
...,...,...,...,...,...,...
2991,ABS,HSX,STOCK,DV Nông nghiệp Bình Thuận,Công ty Cổ phần Dịch vụ Nông nghiệp Bình Thuận,STO
2992,ABR,HSX,STOCK,Đầu tư Nhãn Hiệu Việt,Công ty Cổ phần Đầu tư Nhãn Hiệu Việt,STO
2997,AAT,HSX,STOCK,Tập đoàn Tiên Sơn Thanh Hóa,Công ty Cổ phần Tập đoàn Tiên Sơn Thanh Hóa,STO
2999,AAM,HSX,STOCK,Thủy sản Mekong,Công ty Cổ Phần Thủy Sản MeKong,STO


,sub_company_name,sub_own_percent
0,Viet First Securities Corporation,0.099
1,Tan Binh Real Estate Joint Stock Company,0.055
2,Hai Ha Confectionery Joint Stock Company,0.050
3,Damsan Joint Stock Company,0.050
4,Song Da 505 Joint Stock Company,0.050
5,Central Hydropower Joint Stock Company,0.048
6,Bac Viet Group Joint Stock Company,0.045
7,FECON Corporation,0.011
8,An Binh Commercial Joint Stock Bank,0.000
9,APG Securities Joint Stock Company,0.000


In [9]:
company.officers()

,officer_name,officer_position,officer_own_percent
0,Nguyen Hoang Hai,Member of Management Board,0.1133
1,Nguyen Hoang Hai,Deputy General Director,0.1133
4,Nguyen Viet Anh,Legal Representative,0.0836
3,Nguyen Viet Anh,Chairman of Management Board,0.0836
2,Nguyen Viet Anh,None,0.0836
5,Tran Thi Thu Ha,None,0.0813
6,Le Tu Anh,None,0.0621
7,Nguyen Thi Minh Hien,None,0.0589
8,Nguyen Thi Minh Hien,None,0.0555
9,Le Thi Hoa,None,0.0469


In [10]:
company.events(lang="vi")

,rsi,rs,id,price,price_change,price_change_ratio,price_change_ratio_1m,event_name,event_code,notify_date,exer_date,reg_final_date,exer_right_date,event_desc
0,64.0,NaN,2599156,19000,0,0.000,0.000,XHC - Audited Financial Statement Quarter 2/2025,KQQY,2025-08-18,2025-08-18,1753-01-01,1753-01-01,XHC - Audited Financial Statement Quarter 2/2025
1,64.0,NaN,2595529,19000,0,0.000,0.055,XHC - Financial Statement Quarter 2/2025,KQQY,2025-07-23,2025-07-23,1753-01-01,1753-01-01,XHC - Financial Statement Quarter 2/2025
2,37.5,NaN,2594463,18001,1990,0.124,0.000,XHC - Pay cash dividend 2024 - Year at VND 100...,DIV,2025-07-09,2025-08-01,2025-07-16,2025-07-15,Xuan Hoa Viet Nam Joint Stock Company (XHC) pa...
3,46.4,NaN,2591692,18095,0,0.000,0.016,XHC - Holds 2025 AGM,AGME,2025-05-12,2025-06-20,2025-05-28,2025-05-27,Xuan Hoa Viet Nam Joint Stock Company (XHC) ho...
4,27.7,NaN,2589821,17622,0,0.000,-0.155,XHC - Financial Statement Quarter 1/2025,KQQY,2025-04-23,2025-04-23,1753-01-01,1753-01-01,XHC - Financial Statement Quarter 1/2025
5,29.5,NaN,2588068,17811,0,0.000,-0.005,XHC - Audited Financial Statement FY 2024,KQCT,2025-04-03,2025-04-03,1753-01-01,1753-01-01,XHC - Audited Financial Statement FY 2024
6,29.5,NaN,2592266,17811,0,0.000,-0.005,XHC - Audited Financial Statement Quarter 4/2024,KQQY,2025-04-03,2025-04-03,1753-01-01,1753-01-01,XHC - Audited Financial Statement Quarter 4/2024
7,39.0,NaN,2581698,18948,0,0.000,-0.240,XHC - Financial Statement Quarter 4/2024,KQQY,2025-02-07,2025-02-07,1753-01-01,1753-01-01,XHC - Financial Statement Quarter 4/2024
8,50.3,NaN,2576326,18948,0,0.000,0.000,XHC - Financial Statement Quarter 3/2024,KQQY,2024-10-22,2024-10-22,1753-01-01,1753-01-01,XHC - Financial Statement Quarter 3/2024
9,52.5,NaN,2574667,21885,2463,0.127,0.149,XHC - Audited Financial Statement Quarter 2/2024,KQQY,2024-08-16,2024-08-16,1753-01-01,1753-01-01,XHC - Audited Financial Statement Quarter 2/2024


company2.over

In [30]:
company2.events()

,id,event_title,en__event_title,public_date,issue_date,source_url,event_list_code,ratio,value,record_date,exright_date,event_list_name,en__event_list_name
0,33079,"MBS - Trả cổ tức Cả năm, 2016 bằng tiền 500 VN...","MBS - Pays Year, 2016 cash dividend at VND500/...",2017-08-18,2017-09-25,http://fiinpro.com/News/Detail/578190?lang=vi-VN,DIV,0.0500,500.0,2017-08-25,2017-08-24,Trả cổ tức bằng tiền mặt,Cash Dividend
1,33082,"MBS - Trả cổ tức Cả năm, 2019 bằng tiền 1000 V...","MBS - Pays Year, 2019 cash dividend at VND1000...",2020-06-10,2020-06-24,http://fiinpro.com/News/Detail/5166655?lang=vi-VN,DIV,0.1000,1000.0,2020-06-16,2020-06-15,Trả cổ tức bằng tiền mặt,Cash Dividend
2,41112422,MBS - Trả cổ tức Cả năm năm 2023 bằng tiền 120...,MBS - Pay cash dividend 2023 - Year at VND 120...,2024-07-11,2024-08-23,http://fiinpro.com/News/Detail/11253405?lang=v...,DIV,0.1200,1200.0,2024-07-25,2024-07-24,Trả cổ tức bằng tiền mặt,Cash Dividend
3,60551015,MBS - Trả cổ tức Cả năm năm 2024 bằng tiền 120...,MBS - Pay cash dividend 2024 - Year at VND 120...,2025-08-07,2025-09-19,http://fiinpro.com/News/Detail/11665724?lang=v...,DIV,0.1200,1200.0,2025-08-20,2025-08-19,Trả cổ tức bằng tiền mặt,Cash Dividend
4,33080,MBS- Phát hành cổ phiếu cho cổ đông hiện hữu t...,MBS- Issues additional shares for existing sha...,2020-02-05,2020-02-11,http://fiinpro.com/News/Detail/4407043?lang=vi-VN,ISS,0.2866,0.0,2020-02-12,2020-02-11,Phát hành cổ phiếu,Share Issue
5,33083,MBS - Phát hành cổ phiếu trả cổ tức tỷ lệ 20:3,MBS - Issues dividend shares at 20:3,2021-05-14,2021-05-17,http://fiinpro.com/News/Detail/7648970?lang=vi-VN,ISS,0.1500,0.0,2021-05-18,2021-05-17,Phát hành cổ phiếu,Share Issue
6,11088801,MBS - Phát hành cổ phiếu trả cổ tức tỷ lệ 20%,MBS - Issues dividend shares at 20%,2022-08-09,2022-08-12,http://fiinpro.com/News/Detail/10453518?lang=v...,ISS,0.2000,0.0,2022-08-15,2022-08-12,Phát hành cổ phiếu,Share Issue
7,15420283,"MBS - Niêm yết bổ sung 112,974,794 cổ phiếu","MBS- Lists additional 112,974,794 shares",2022-11-22,2022-11-17,http://fiinpro.com/News/Detail/10605917?lang=v...,AIS,0.0000,0.0,1753-01-01,1753-01-01,Niêm yết thêm,Additional Listing
8,26045990,MBS - Phát hành cổ phiếu trả cổ tức tỷ lệ 12%,MBS - Issues dividend shares at 12%,2023-08-07,2023-08-10,http://fiinpro.com/News/Detail/10887192?lang=v...,ISS,0.1200,0.0,2023-08-11,2023-08-10,Phát hành cổ phiếu,Share Issue
9,33084,MBS - Niêm yết bổ sung 93.959.939 cổ phiếu,"MBS - Lists additional 93,959,939 shares",2021-07-20,2021-07-26,http://fiinpro.com/News/Detail/8208209?lang=vi-VN,AIS,0.0000,0.0,1753-01-01,1753-01-01,Niêm yết thêm,Additional Listing


In [12]:
listing.all_symbols()



,symbol,organ_name
0,YTC,Công ty Cổ phần Xuất nhập khẩu Y tế Thành phố ...
1,YEG,Công ty Cổ phần Tập đoàn Yeah1
2,YBM,Công ty Cổ phần Khoáng sản Công nghiệp Yên Bái
3,YBC,Công ty Cổ phần Xi măng và Khoáng sản Yên Bái
4,XPH,Công ty Cổ phần Xà phòng Hà Nội
...,...,...
1714,AAS,Công ty Cổ phần Chứng khoán SmartInvest
1715,AAM,Công ty Cổ Phần Thủy Sản MeKong
1716,AAH,Công ty Cổ phần Hợp Nhất
1717,AAA,Công ty Cổ phần Nhựa An Phát Xanh


In [10]:
company2.subsidiaries()

RetryError: RetryError[<Future at 0x27363aee890 state=finished raised KeyError>]

In [26]:
listing.symbols_by_industries()

,symbol,organ_name,icb_name3,icb_name2,icb_name4,com_type_code,icb_code1,icb_code2,icb_code3,icb_code4
0,F88,Công ty Cổ phần Đầu tư F88,Dịch vụ tài chính,Dịch vụ tài chính,Tài chính cá nhân,CT,8000,8700,8770,8773
1,DVT,Trung tâm Đào tạo nghiệp vụ Giao thông vận tải...,Tư vấn & Hỗ trợ Kinh doanh,Hàng & Dịch vụ Công nghiệp,Đào tạo & Việc làm,CT,2000,2700,2790,2793
2,AFX,Công ty Cổ phần Xuất Nhập khẩu Nông sản Thực p...,Phân phối thực phẩm & dược phẩm,Bán lẻ,Phân phối thực phẩm,CT,5000,5300,5330,5337
3,AGR,Công ty Cổ phần Chứng khoán AGRIBANK,Dịch vụ tài chính,Dịch vụ tài chính,Môi giới chứng khoán,CK,8000,8700,8770,8777
4,ALT,Công ty Cổ phần Văn hóa Tân Bình,Truyền thông,Truyền thông,"Sách, ấn bản & sản phẩm văn hóa",CT,5000,5500,5550,5557
...,...,...,...,...,...,...,...,...,...,...
1579,XMC,Công ty cổ phần Đầu tư và Xây dựng Xuân Mai,Xây dựng và Vật liệu,Xây dựng và Vật liệu,Vật liệu xây dựng & Nội thất,CT,2000,2300,2350,2353
1580,XMD,Công ty Cổ phần Xuân Mai - Đạo Tú,Xây dựng và Vật liệu,Xây dựng và Vật liệu,Vật liệu xây dựng & Nội thất,CT,2000,2300,2350,2353
1581,CQT,Công ty Cổ phần Xi măng Quán Triều VVMI,Xây dựng và Vật liệu,Xây dựng và Vật liệu,Vật liệu xây dựng & Nội thất,CT,2000,2300,2350,2353
1582,XPH,Công ty Cổ phần Xà phòng Hà Nội,Hàng cá nhân,Hàng cá nhân & Gia dụng,Hàng cá nhân,CT,3000,3700,3760,3767


In [1]:
company.subsidiaries()

NameError: name 'company' is not defined

In [3]:
company.subsidiaries()

,sub_company_name,sub_own_percent
0,Vitaco Danang Company Limited,1.0
1,Vitaco Saigon Company Limited,1.0
2,Vitaco Services Trading Company Limited,1.0
